In [2]:
import streamlit as st
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [3]:
conn = sqlite3.connect('../Base de datos SQLite/siniestros_viales.db')

In [4]:
hechos_df = pd.read_sql_query('SELECT * FROM hechos', conn)
victimas_df = pd.read_sql_query('SELECT * FROM victimas', conn)

In [6]:
hechos_df.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01 00:00:00,2016,1,1,04:00:00.000000,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02 00:00:00,2016,1,2,01:15:00.000000,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03 00:00:00,2016,1,3,07:00:00.000000,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,None,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10 00:00:00,2016,1,10,00:00:00.000000,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21 00:00:00,2016,1,21,05:20:00.000000,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [7]:
victimas_df.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01 00:00:00,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02 00:00:00,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03 00:00:00,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10 00:00:00,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21 00:00:00,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [8]:
conn.close()

In [10]:
# Eliminar las columnas ALTURA, CRUCE y DIRECCION_NORMALIZADA
hechos_df.drop(columns=['Altura', 'Cruce', 'Dirección Normalizada'], inplace=True)

In [11]:
# Combinar los datos de hechos y víctimas
combined_df = pd.merge(hechos_df, victimas_df, left_on='ID', right_on='ID_hecho', how='left')

In [14]:
# Convertir las columnas 'POS_X' y 'POS_Y' a valores numéricos, manejando errores
combined_df['pos x'] = pd.to_numeric(combined_df['pos x'], errors='coerce')
combined_df['pos y'] = pd.to_numeric(combined_df['pos y'], errors='coerce')

In [15]:
# Limpiar la columna 'EDAD' convirtiéndola a valores numéricos, manejando errores
combined_df['EDAD'] = pd.to_numeric(combined_df['EDAD'], errors='coerce')

In [16]:
# Eliminar los valores nulos del dataframe combinado
combined_df.dropna(inplace=True) 

In [28]:
# Renombrar las columnas 'POS_Y' y 'POS_X' a 'latitude' y 'longitude'
combined_df.rename(columns={'pos y': 'latitude', 'pos x': 'longitude'}, inplace=True)

In [29]:
combined_df

,ID,N_VICTIMAS,FECHA_x,AAAA_x,MM_x,DD_x,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,ID_hecho,FECHA_y,AAAA_y,MM_y,DD_y,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01 00:00:00,2016,1,1,04:00:00.000000,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,2016-0001,2016-01-01 00:00:00,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02 00:00:00,2016,1,2,01:15:00.000000,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,2016-0002,2016-01-02 00:00:00,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03 00:00:00,2016,1,3,07:00:00.000000,7,AV ENTRE RIOS 2034,AVENIDA,...,2016-0003,2016-01-03 00:00:00,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10 00:00:00,2016,1,10,00:00:00.000000,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,2016-0004,2016-01-10 00:00:00,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,SD
4,2016-0005,1,2016-01-21 00:00:00,2016,1,21,05:20:00.000000,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,2016-0005,2016-01-21 00:00:00,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13 00:00:00,2021,12,13,17:10:00.000000,17,AV. RIESTRA Y MOM,AVENIDA,...,2021-0093,2021-12-13 00:00:00,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18.0,2021-12-18 00:00:00
713,2021-0094,1,2021-12-20 00:00:00,2021,12,20,01:10:00.000000,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,2021-0094,2021-12-20 00:00:00,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43.0,2021-12-20 00:00:00
714,2021-0095,1,2021-12-30 00:00:00,2021,12,30,00:43:00.000000,0,AV. GAONA Y TERRADA,AVENIDA,...,2021-0095,2021-12-30 00:00:00,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27.0,2022-01-02 00:00:00
715,2021-0096,1,2021-12-15 00:00:00,2021,12,15,10:30:00.000000,10,AV. EVA PERON 4071,AVENIDA,...,2021-0096,2021-12-15 00:00:00,2021,12,15,CONDUCTOR,AUTO,MASCULINO,60.0,2021-12-20 00:00:00


In [30]:
# Función para cargar y combinar los datos desde la base de datos SQLite
def load_data():
    conn = sqlite3.connect('../Base de datos SQLite/siniestros_viales.db')
    
    hechos_df = pd.read_sql_query('SELECT * FROM hechos', conn)
    victimas_df = pd.read_sql_query('SELECT * FROM victimas', conn)
    
    conn.close()
    
    # Eliminar las columnas ALTURA, CRUCE y DIRECCION_NORMALIZADA
    hechos_df.drop(columns=['Altura', 'Cruce', 'Dirección Normalizada'], inplace=True)
    
    # Combinar los datos de hechos y víctimas
    combined_df = pd.merge(hechos_df, victimas_df, left_on='ID', right_on='ID_hecho', how='left')
    
    # Convertir las columnas 'POS_X' y 'POS_Y' a valores numéricos, manejando errores
    combined_df['pos x'] = pd.to_numeric(combined_df['pos x'], errors='coerce')
    combined_df['pos y'] = pd.to_numeric(combined_df['pos y'], errors='coerce')
    
    # Limpiar la columna 'EDAD' convirtiéndola a valores numéricos, manejando errores
    combined_df['EDAD'] = pd.to_numeric(combined_df['EDAD'], errors='coerce')
    
    # Eliminar los valores nulos del dataframe combinado
    combined_df.dropna(inplace=True)

    # Renombrar las columnas 'POS_Y' y 'POS_X' a 'latitude' y 'longitude'
    combined_df.rename(columns={'pos y': 'latitude', 'pos x': 'longitude'}, inplace=True)
    
    return combined_df


In [31]:
# Cargar los datos
data = load_data()

In [32]:
# Crear el dashboard con Streamlit
st.title('Dashboard de Siniestros Viales en CABA')

DeltaGenerator()

In [33]:
# Filtros de selección
st.sidebar.header('Filtros')
anio = st.sidebar.selectbox('Año', sorted(data['AAAA_x'].unique()))
mes = st.sidebar.selectbox('Mes', sorted(data['MM_x'].unique()))

In [34]:
# Filtrar los datos según los filtros seleccionados
filtered_data = data[(data['AAAA_x'] == anio) & (data['MM_x'] == mes)]

In [35]:
# KPI 1: Tasa de homicidios en siniestros viales
poblacion_total = 3075646  # Población de CABA
num_homicidios = filtered_data['ID_hecho'].nunique()
tasa_homicidios = (num_homicidios / poblacion_total) * 100000
st.metric('Tasa de Homicidios en Siniestros Viales', f'{tasa_homicidios:.2f} por 100,000 habitantes')

DeltaGenerator()

In [36]:
# KPI 2: Cantidad de accidentes mortales de motociclistas
num_accidentes_motos = filtered_data[filtered_data['VICTIMA_y'] == 'MOTO']['ID_hecho'].nunique()
st.metric('Accidentes Mortales de Motociclistas', num_accidentes_motos)

DeltaGenerator()

In [37]:
# Visualización de los datos
st.subheader('Mapa de Siniestros Viales')
st.map(filtered_data[['latitude', 'longitude']])

DeltaGenerator()

In [38]:
st.subheader('Distribución de Homicidios por Edad y Sexo')
fig, ax = plt.subplots()
filtered_data['EDAD'].hist(ax=ax, bins=30)
ax.set_xlabel('Edad')
ax.set_ylabel('Cantidad')
st.pyplot(fig)

DeltaGenerator()

In [39]:
st.subheader('Distribución de Homicidios por Edad y Sexo')
fig, ax = plt.subplots()
filtered_data['EDAD'].hist(ax=ax, bins=30)
ax.set_xlabel('Edad')
ax.set_ylabel('Cantidad')
st.pyplot(fig)

DeltaGenerator()

In [40]:
st.subheader('Distribución de Homicidios por Edad y Sexo')
fig, ax = plt.subplots()
filtered_data['EDAD'].hist(ax=ax, bins=30)
ax.set_xlabel('Edad')
ax.set_ylabel('Cantidad')
st.pyplot(fig)

DeltaGenerator()

In [42]:
st.subheader('Homicidios por Tipo de Participante')
fig, ax = plt.subplots()
filtered_data['VICTIMA_y'].value_counts().plot(kind='bar', ax=ax)
ax.set_xlabel('Tipo de Participante')
ax.set_ylabel('Cantidad')
st.pyplot(fig)

DeltaGenerator()